In [10]:
import numpy as np
import os
import pickle
from plotting_utils import heatmap,annotate_heatmap
list_of_tasks = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d colorization \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point \
segmentsemantic class_1000 class_places inpainting_whole'

task_list = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d  \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point_well_defined \
segmentsemantic_rb class_1000'

task_list_refined = 'autoencoder curvature denoise edge2d edge3d \
keypoint2d keypoint3d  \
reshade rgb2depth rgb2mist rgb2sfnorm \
room_layout segment25d segment2d vanishing_point \
semantic_seg class_1000'

task_list = task_list.split()
task_list_refined = task_list_refined.split()

with open('./../all_affinities_tk.pkl', 'rb') as f:
    data = pickle.load(f)
    
t_affinity_matrix = np.zeros((len(task_list),len(task_list)))
for i,task1 in enumerate(task_list):
    for j,task2 in enumerate(task_list):
        t_affinity_matrix[i,j] = data[task1+"__"+task2]
        
with open('./../wins_vs_pixels_16k.pkl', 'rb') as f:
    data = pickle.load(f)
#print(data)
#print(data['win_rates']['segmentsemantic_rb'])
import numpy as np
t_winrate_matrix = np.zeros((len(task_list),len(task_list)))
for i,task1 in enumerate(task_list):
    for j,task2 in enumerate(task_list):
        t_winrate_matrix[i,j] = data[task1+"__"+task2]
print(t_winrate_matrix.shape)

(17, 17)


In [27]:
results_path = './../results/DDScomparison_taskonomy/pascal_5000/'

In [28]:
from scipy.stats import spearmanr,pearsonr

kernel_affinities =  np.load(results_path+'kernels.npy',allow_pickle=True).item()
rdm_affinities = np.load(results_path+'rdms.npy',allow_pickle=True).item()

corr_value = {}
for kernel,kernel_affinity in kernel_affinities.items():
    corr_value[kernel] = {}
    for f_ablation,f_ablation_affinity in kernel_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_winrate_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] = temp
        corr_value[kernel][f_ablation] = corr_value_matrix
            

for rdm,rdm_affinity in rdm_affinities.items():
    corr_value[rdm] = {}
    for f_ablation,f_ablation_affinity in rdm_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_winrate_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] =  temp
        corr_value[rdm][f_ablation] = corr_value_matrix

In [29]:
max_corr = 0
mean_corr = {}
for dist,corr_dist in corr_value.items():
    mean_corr[dist] = {}
    for f_ablation,corr_f_ablation in corr_dist.items():
        mean_corr[dist][f_ablation] = round(np.mean(corr_f_ablation),3)
        if np.mean(corr_f_ablation) > max_corr:
            max_corr = np.mean(corr_f_ablation)
            max_combo = [dist,f_ablation]
print(max_corr,max_combo)

0.7843198006864169 ['euclidean', 'znorm']


In [30]:
import pandas as pd
dfObj = pd.DataFrame(mean_corr)
print(dfObj)

                 rbf    lap  linear  pearson  euclidean  cosine
None           0.748  0.748   0.442    0.737      0.556   0.718
centering      0.748  0.748   0.442    0.703      0.556   0.700
znorm          0.783  0.779   0.772    0.769      0.784   0.779
group_norm     0.717  0.736   0.740    0.740      0.735   0.740
instance_norm  0.751  0.760   0.776    0.776      0.771   0.776
layer_norm     0.714  0.739   0.737    0.737      0.733   0.737
batch_norm     0.758  0.768   0.550    0.754      0.683   0.751


In [31]:
mean_col = 0
mean_row = 0
for j,task in enumerate(task_list):
    temp_col,_ = spearmanr(t_winrate_matrix[:,j], t_affinity_matrix[:,j])
    temp_row,_ = spearmanr(t_winrate_matrix[j,:], t_affinity_matrix[j,:])
    mean_col+=temp_col/len(task_list)
    mean_row+=temp_row/len(task_list)
print(mean_col,mean_row)

0.9194340578036585 0.6765866545500134


In [32]:
from scipy.stats import spearmanr,pearsonr

kernel_affinities =  np.load(results_path+'kernels.npy',allow_pickle=True).item()
rdm_affinities = np.load(results_path+'rdms.npy',allow_pickle=True).item()

corr_value = {}
for kernel,kernel_affinity in kernel_affinities.items():
    corr_value[kernel] = {}
    for f_ablation,f_ablation_affinity in kernel_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_affinity_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] = temp
        corr_value[kernel][f_ablation] = corr_value_matrix
            

for rdm,rdm_affinity in rdm_affinities.items():
    corr_value[rdm] = {}
    for f_ablation,f_ablation_affinity in rdm_affinity.items():
        corr_value_matrix = np.zeros(len(task_list))
        temp_affinity = np.delete(f_ablation_affinity, (7, 18,19), axis=0)
        temp_affinity = np.delete(temp_affinity, (7, 18,19), axis=1)
        for j,task in enumerate(task_list):
            temp,_ = spearmanr(t_affinity_matrix[:,j], temp_affinity[:,j])
            corr_value_matrix[j] =  temp
        corr_value[rdm][f_ablation] = corr_value_matrix

In [33]:
max_corr = 0
mean_corr = {}
for dist,corr_dist in corr_value.items():
    mean_corr[dist] = {}
    for f_ablation,corr_f_ablation in corr_dist.items():
        mean_corr[dist][f_ablation] = round(np.mean(corr_f_ablation),3)
        if np.mean(corr_f_ablation) > max_corr:
            max_corr = np.mean(corr_f_ablation)
            max_combo = [dist,f_ablation]
print(max_corr,max_combo)

0.8167531718569782 ['pearson', 'znorm']


In [34]:
import pandas as pd
dfObj = pd.DataFrame(mean_corr)
print(dfObj)

                 rbf    lap  linear  pearson  euclidean  cosine
None           0.778  0.784   0.455    0.784      0.566   0.762
centering      0.778  0.784   0.455    0.748      0.566   0.743
znorm          0.809  0.812   0.809    0.817      0.817   0.816
group_norm     0.755  0.777   0.787    0.787      0.778   0.787
instance_norm  0.784  0.792   0.815    0.814      0.808   0.814
layer_norm     0.755  0.779   0.784    0.784      0.777   0.784
batch_norm     0.790  0.803   0.562    0.807      0.701   0.796
